In [ ]:
# Create working folders
!mkdir -p data/GSE270436
!mkdir -p data/GSE152183

# Download bulk RNA-seq files
!wget -O data/GSE270436/rawCount.txt.gz https://ftp.ncbi.nlm.nih.gov/geo/series/GSE270nnn/GSE270436/suppl/GSE270436_HOMER.rawCount.txt.gz
!wget -O data/GSE270436/rawTPM.txt.gz https://ftp.ncbi.nlm.nih.gov/geo/series/GSE270nnn/GSE270436/suppl/GSE270436_HOMER.rawTPM.txt.gz

# Unzip
!gunzip data/GSE270436/rawCount.txt.gz
!gunzip data/GSE270436/rawTPM.txt.gz

# Download scRNA-seq raw archive
!wget -O data/GSE152183/GSE152183_RAW.tar https://ftp.ncbi.nlm.nih.gov/geo/series/GSE152nnn/GSE152183/suppl/GSE152183_RAW.tar

# Extract it
!tar -xvf data/GSE152183/GSE152183_RAW.tar -C data/GSE152183

In [ ]:
import pandas as pd

counts = pd.read_csv("data/GSE270436/rawCount.txt", sep="\t", index_col=0)
counts.head()

In [ ]:
sample_columns = counts.columns[5:]
print(len(sample_columns))
print(sample_columns.tolist())

In [ ]:
samples = sample_columns.tolist()

def assign_group(sample):
    matches = []

    # Control samples
    if any(k in sample for k in ["42N", "D4N", "71N"]):
        matches.append("Control")

    # MHS samples
    if any(k in sample for k in ["1F37", "1G4317"]):
        matches.append("MHS")

    # KO samples (be explicit!)
    if any(k in sample for k in [
        "2H10131", "2H10", "2H11", "2H9", "237", "rep237"
    ]):
        matches.append("KO")

    if len(matches) == 1:
        return matches[0]
    elif len(matches) == 0:
        raise ValueError(f"No group match for sample: {sample}")
    else:
        raise ValueError(f"Ambiguous group match for sample: {sample} → {matches}")

meta = pd.DataFrame({
    "sample": samples,
    "group": [assign_group(s) for s in samples]
})

# validation
print(meta["group"].value_counts())
assert set(meta["group"]) == {"Control", "MHS", "KO"}

In [ ]:
!pip install --upgrade pip
!pip install pydeseq2

In [ ]:
import pydeseq2
print(pydeseq2.__version__)

In [ ]:
# Select only real sample columns (skip first 5 annotation columns)
sample_cols = counts.columns[5:]


# Build gene-linked regulatory activity matrix
expr = (
    counts
    .groupby("Gene", as_index=True)[sample_cols]
    .sum()
)

# sanity check
assert expr.index.is_unique

expr.shape, meta.shape

In [ ]:
print("Unique genes:", expr.index.is_unique)
print(meta["group"].value_counts())
print(set(expr.columns) == set(meta["sample"]))

In [ ]:
expr.index.is_unique

In [ ]:
expr.shape, meta.shape

In [ ]:
set(expr.columns) == set(meta["sample"])

In [ ]:
# Select only sample columns
sample_cols = counts.columns[5:]

# Group by Gene and SUM counts across peaks
expr = (
    counts
    .groupby("Gene")[sample_cols]
    .sum()
)

expr.shape

In [ ]:
expr.index.is_unique

In [ ]:
set(expr.columns) == set(meta["sample"])

In [ ]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

dds = DeseqDataSet(
    counts=expr.T,                      # samples × genes
    metadata=meta.set_index("sample"),
    design="~ group",
    refit_cooks=True
)

dds.deseq2()

In [ ]:
stat_KO = DeseqStats(dds, contrast=("group", "KO", "Control"))
stat_KO.summary()
res_KO = stat_KO.results_df

In [ ]:
stat_MHS = DeseqStats(dds, contrast=("group", "MHS", "Control"))
stat_MHS.summary()
res_MHS = stat_MHS.results_df

In [ ]:
deg_KO = res_KO[(res_KO["padj"] < 0.05) & (abs(res_KO["log2FoldChange"]) > 1)]
deg_MHS = res_MHS[(res_MHS["padj"] < 0.05) & (abs(res_MHS["log2FoldChange"]) > 1)]

deg_KO.shape, deg_MHS.shape

In [ ]:
deg_KO.to_csv("DEG_KO_vs_Control.csv")
deg_MHS.to_csv("DEG_MHS_vs_Control.csv")

In [ ]:
# Drop NA genes
res_KO = res_KO.dropna()
res_MHS = res_MHS.dropna()

# Regulatory-appropriate thresholds
deg_KO = res_KO[
    (res_KO["padj"] < 0.05) &
    (abs(res_KO["log2FoldChange"]) > 0.3)
]

deg_MHS = res_MHS[
    (res_MHS["padj"] < 0.05) &
    (abs(res_MHS["log2FoldChange"]) > 0.5)
]

deg_KO.shape, deg_MHS.shape

In [ ]:
(res_KO["padj"] < 0.05).sum(), (res_MHS["padj"] < 0.05).sum()

In [ ]:
mhs_sig = res_MHS[res_MHS["padj"] < 0.05]
len(mhs_sig)

In [ ]:
!pip install gseapy

In [ ]:
mhs_genes = mhs_sig.index.tolist()
len(mhs_genes)

In [ ]:
import gseapy as gp

enr_go = gp.enrichr(
    gene_list=mhs_genes,
    gene_sets="GO_Biological_Process_2021",
    organism="Human",
    outdir=None
)

enr_go.results.head(20)

In [ ]:
!mkdir -p data/GSE152183
!cd data/GSE152183

In [ ]:
!wget -O GSE152183_RAW.tar \
https://ftp.ncbi.nlm.nih.gov/geo/series/GSE152nnn/GSE152183/suppl/GSE152183_RAW.tar

In [ ]:
!tar -xvf GSE152183_RAW.tar

In [ ]:
ls *_matrix.mtx.gz | wc -l

In [ ]:
def load_sample(prefix):
    adata = sc.read_mtx(f"{prefix}_matrix.mtx.gz").T

    genes = pd.read_csv(f"{prefix}_features.tsv.gz", sep="\t", header=None)
    barcodes = pd.read_csv(f"{prefix}_barcodes.tsv.gz", sep="\t", header=None)

    adata.var_names = genes[1].values
    adata.obs_names = barcodes[0].values

    adata.var_names_make_unique()
    return adata

In [ ]:
import os

samples = []

for f in os.listdir("."):
    if f.endswith("_matrix.mtx.gz"):
        prefix = f.replace("_matrix.mtx.gz", "")
        adata = load_sample(prefix)

        adata.obs["method"] = "Dounce" if "Dounce" in prefix else "Miltenyi"
        adata.obs["incubation"] = "IN" if "IN" in prefix else "NO"
        adata.obs["stress"] = adata.obs["method"] + "_" + adata.obs["incubation"]

        samples.append(adata)

len(samples)

In [ ]:
!pip install scanpy anndata scipy

In [ ]:
import scanpy as sc
import pandas as pd
import os

In [ ]:
adata = sc.concat(samples, label="batch", index_unique="-")
adata

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=50)

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
adata.obs["stress_level"] = "mid"

adata.obs.loc[
    (adata.obs["method"] == "Miltenyi") & (adata.obs["incubation"] == "NO"),
    "stress_level"
] = "low"

adata.obs.loc[
    (adata.obs["method"] == "Dounce") & (adata.obs["incubation"] == "IN"),
    "stress_level"
] = "high"

adata.obs["stress_level"].value_counts()

In [ ]:
sc.tl.rank_genes_groups(
    adata,
    groupby="stress_level",
    groups=["high"],
    reference="low",
    method="wilcoxon"
)

In [ ]:
stress_df = sc.get.rank_genes_groups_df(adata, group="high")
stress_df.head()

In [ ]:
stress_df.sort_values("pvals_adj").head(15)

In [ ]:
stress_genes = stress_df[
    (stress_df["pvals_adj"] < 0.05) &
    (stress_df["logfoldchanges"].abs() > 0.25)
]["names"].unique().tolist()

len(stress_genes)

In [ ]:
clean_mhs = mhs_sig.drop(index=set(stress_genes), errors="ignore")

len(mhs_sig), len(clean_mhs)

In [ ]:
set(mhs_sig.index) & set(stress_genes)

In [ ]:
stress_genes_upper = [g.upper() for g in stress_genes]
set(mhs_sig.index) & set(stress_genes_upper)

In [ ]:
len(set(mhs_sig.index) & set(stress_genes_upper))

In [ ]:
clean_mhs = mhs_sig.drop(index=set(stress_genes_upper), errors="ignore")

In [ ]:
import gseapy as gp

enr_stress = gp.enrichr(
    gene_list=stress_genes,
    gene_sets="GO_Biological_Process_2021",
    organism="Mouse",
    outdir=None
)

stress_go = enr_stress.results.sort_values("Adjusted P-value")
stress_go.head(20)

In [ ]:
print("MHS sig :", len(mhs_sig))
print("Clean MHS:", len(clean_mhs))

# prove stress genes are gone
set(clean_mhs.index) & set(stress_genes_upper)

In [ ]:
mhs_genes_raw   = mhs_sig.index.tolist()
mhs_genes_clean = clean_mhs.index.tolist()

len(mhs_genes_raw), len(mhs_genes_clean)

In [ ]:
enr_mhs = gp.enrichr(
    gene_list=mhs_genes_raw,
    gene_sets="GO_Biological_Process_2021",
    organism="Human",
    outdir=None
)

mhs_go = enr_mhs.results.sort_values("Adjusted P-value")
mhs_go.head(20)

In [ ]:
stress_terms = set(stress_go["Term"])

mhs_go["is_stress_related"] = mhs_go["Term"].isin(stress_terms)

mhs_go[["Term", "Adjusted P-value", "is_stress_related"]].head(20)

In [ ]:
mhs_go["is_stress_related"] = mhs_go["Term"].isin(stress_terms)

In [ ]:
mhs_go[["Term", "Adjusted P-value", "is_stress_related"]].head(30)

In [ ]:
clean_mhs_go = mhs_go[mhs_go["is_stress_related"] == False]
clean_mhs_go.head(20)

In [ ]:
len(mhs_go), len(clean_mhs_go)

In [ ]:
!pip install --user scikit-misc


In [ ]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    flavor="cell_ranger",
    subset=True
)

In [ ]:
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver="arpack")
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=30)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(
    adata,
    color="stress_level",
    palette={
        "low": "#2ca02c",   # green
        "mid": "#bdbdbd",   # gray
        "high": "#d62728"   # red
    },
    size=5,
    frameon=False,
    title="Dissociation stress separates microglial states",
    save="_stress_level.pdf"
)

In [ ]:
adata.obs["method"].value_counts()

In [ ]:
adata.obs.groupby(["method", "batch"]).size()

In [ ]:
sc.pl.umap(
    adata,
    color="method",
    palette="Set2",
    size=5,
    frameon=False,
    title="Microglia dissociation methods",
    save="_dissociation_method.pdf"
)

In [ ]:
stress_genes_plot = [
    "FOS", "JUN", "JUNB", "HSPA1A", "DUSP1", "IER5"
]

mef2c_genes_plot = [
    "TREM2", "CD36", "CTSS", "TCIRG1",
    "CERS6", "SGMS1", "ASAH1"
]

In [ ]:
adata.var_names[:20]

In [ ]:
adata.var.head(5)

In [ ]:
stress_genes_plot = [
    "Fos", "Jun", "Junb", "Hspa1a", "Dusp1", "Ier5"
]

valid_stress = [g for g in stress_genes_plot if g in adata.var_names]
valid_stress

In [ ]:
sc.tl.score_genes(adata, valid_stress, score_name="Stress_score")

In [ ]:
mef2c_genes_plot = [
    "Trem2", "Cd36", "Ctss", "Tcirg1",
    "Cers6", "Sgms1", "Asah1"
]

valid_mef2c = [g for g in mef2c_genes_plot if g in adata.var_names]
valid_mef2c

In [ ]:
sc.tl.score_genes(adata, valid_mef2c, score_name="MEF2C_score")

In [ ]:
# Use top stress genes by adjusted p-value
stress_genes_scoring = (
    stress_df
    .sort_values("pvals_adj")
    .query("pvals_adj < 0.01")
    .head(30)["names"]
    .tolist()
)

# ensure present
stress_genes_scoring = [g for g in stress_genes_scoring if g in adata.var_names]
len(stress_genes_scoring)

In [ ]:
sc.tl.score_genes(
    adata,
    stress_genes_scoring,
    score_name="Stress_score"
)

In [ ]:
adata.obs.groupby("stress_level")["Stress_score"].mean()

In [ ]:
sc.pl.umap(
    adata,
    color=["Stress_score", "MEF2C_score"],
    cmap="viridis",
    size=5,
    frameon=False,
    wspace=0.35,
    save="_stress_vs_mef2c_modules.png"
)

In [ ]:
# Stress genes actually used
print("Stress genes used:", valid_stress)
print("N stress genes used:", len(valid_stress))

# MEF2C genes actually used
print("MEF2C genes used:", valid_mef2c)
print("N MEF2C genes used:", len(valid_mef2c))

In [ ]:
for s in ["Stress_score", "MEF2C_score"]:
    print(
        s,
        "NaNs:", adata.obs[s].isna().sum(),
        "Std:", adata.obs[s].std(),
        "Min/Max:", adata.obs[s].min(), adata.obs[s].max()
    )

In [ ]:
adata.obs.groupby("stress_level")["Stress_score"].mean()